In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import sklearn

In [3]:
meme = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_meme.csv')
event = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_event.csv')
user = pd.read_csv('drive/My Drive/memeproject/Danila FFmemes/danila_user.csv')

In [7]:
meme.head()

,meme_id,meme_type,source_id,published_at,caption,content_lang,is_available,likes,dislikes,created_at,updated_at,is_broken,content_text,caption_lang,language_id
0,28997,photo,79.0,2020-04-12 01:48:13+00,I mean shitt it ain't their fault,en,True,17,14,2020-04-12 09:04:37.341908+00,2020-10-26 19:20:58.198825+00,False,brandon @brndvx females live on recruit diffic...,en,3.0
1,10873,photo,43.0,2020-03-25 11:48:14+00,NaN,ru,True,79,74,2020-03-25 20:00:01.154096+00,2020-10-26 19:20:58.226846+00,False,"Я не робот Докажите, что всякое односвязное Ит...",ww,2.0
2,9874,photo,78.0,2020-03-24 07:18:03+00,Anime toddies expert,en,True,1,7,2020-03-24 13:06:18.588359+00,2020-10-26 19:20:58.259045+00,False,When you find the actual interview and it does...,en,3.0
3,43458,photo,43.0,2020-05-23 12:50:52+00,NaN,ru,False,8,12,2020-05-24 22:03:34.926116+00,2020-10-26 19:20:58.265179+00,False,Google сколько калорий в грамме урана а Все В ...,ww,2.0
4,9000,photo,76.0,2020-03-23 07:16:05+00,New treasure,und,True,2,9,2020-03-23 13:04:55.258554+00,2020-10-26 19:20:58.29742+00,False,YRO .20.,en,3.0


In [8]:
event.head()

,event_id,reaction_id,session_idx,meme_idx,created_at,updated_at,meme_id,user_id
0,3607617,2.0,55,11,2020-10-24 19:45:50.964474+00,2020-10-24 19:45:51.049752+00,24379,280747086
1,3608100,1.0,280,45,2020-10-24 19:52:33.681933+00,2020-10-24 19:52:33.763993+00,49173,285739883
2,3607629,1.0,55,13,2020-10-24 19:46:08.690407+00,2020-10-24 19:46:08.779079+00,8340,280747086
3,3607649,NaN,30,2,2020-10-24 19:46:24.443014+00,2020-10-24 19:46:24.519787+00,64748,78991522
4,3608045,2.0,35,341,2020-10-24 19:51:28.544856+00,2020-10-24 19:51:28.586788+00,45163,860086058


In [9]:
user.head()

,user_id,deep_link,is_blocked_bot,is_banned,is_moderator,sessions,last_session_memes,watched_memes,created_at,updated_at,interface_lang,gender
0,820326025,anime0ch_lang_ru,False,False,False,6,1,20,2020-09-12 20:07:11.668743+00,2020-10-26 12:16:18.458654+00,ru,M
1,1040875164,likemeupbot_lang_en,False,False,False,2,1,7,2020-07-21 05:54:12.671036+00,2020-10-26 14:37:05.637591+00,en,NaN
2,449432539,debil_ebaniy,True,False,False,2,1,9,2020-09-14 19:02:06.80911+00,2020-10-17 21:26:52.029824+00,ru,F
3,230248850,NaN,False,False,False,26,1,38,2020-03-16 00:36:42.252438+00,2020-10-26 12:11:02.159405+00,ru,M
4,7719424,NaN,True,False,False,27,6,49,2020-03-15 22:30:45.300242+00,2020-10-17 21:26:52.19941+00,ru,M


## Reaction handling

Removing broken memes

In [4]:
meme = meme.loc[meme.is_broken==False]

Public likes

In [5]:
source_like = meme[['source_id','likes']].groupby('source_id').sum()
source_like.head()

,likes
source_id,
1.0,141
2.0,8753
3.0,17852
4.0,17265
5.0,11334


Public dislikes

In [6]:
source_dislike = meme[['source_id','dislikes']].groupby('source_id').sum()
source_dislike.head()

,dislikes
source_id,
1.0,447
2.0,11876
3.0,19303
4.0,27841
5.0,15323


Meme to etself source

In [10]:
event = pd.merge(event, meme[['meme_id','source_id']], on='meme_id')

How many likes did user give to the source?

In [11]:
event.loc[event.reaction_id == 3, 'reaction_id'] = 1
user_source_like = event.loc[event.reaction_id == 1][['user_id', 'source_id','reaction_id']].groupby(['user_id', 'source_id']).count()
user_source_like.head()

reaction_id
user_id source_id             
1615    8.0                  1
        11.0                 1
        13.0                 2
        18.0                 1
        25.0                 1

How many dislikes did user give to the source?

In [12]:
event.loc[event.reaction_id == 4, 'reaction_id'] = 2
user_source_dislike = event.loc[event.reaction_id == 2][['user_id', 'source_id','reaction_id']].groupby(['user_id', 'source_id']).count()
user_source_dislike.head()

reaction_id
user_id source_id             
1615    3.0                  2
        15.0                 2
        20.0                 2
        27.0                 1
        30.0                 1

# План по рек системе

## Постановка задачи

На вход дается пара (мем, юзер). Задача бинарной классификации: на выход нужно предсказать, понравится (1) ли мем юзеру или нет (0). Можно предсказать вероятность.

## Данные

Есть таблички events, memes, user-source (тут суперлайк считается за 5 лайков, а удалить считается за 5 дизлайков).
Можно делать какую угодно рекомендательную систему.

## Анализ данных

Список интересных вопросов:

• В какой момент юзеры отваливаются
• Отличается ли поведение англ/ру юзеров, статистика англ/ру мемов


Ниже "классический" вариант фич, не учитывающий в явном виде user_id и meme_id:

• сумма лайков мема
• сумма дизлайков мема
• сумма лайков паблика, из которого спарсен мем
• сумма дизлайков паблика
• сумма лайков пары юзер-паблик
• сумма дизлайков пары юзер-паблик
• фичи мема: content language, caption language, ...
• фичи юзера: язык, origin, ...
+ инжинирить новые фичи типа общего кол-ва реакций, лайк-рейта (сумма лайков мема на сумму всех реакций)
+ сумма суперлайков, супердизлайков и проч

После чего применяем какой-нибудь gradient boosting из коробки (или любую другую модель из sklearn).

## Нюансы

• Использовать user_id и meme_id как категориальные признаки
• Если тренить модель отдельно на каждого юзера, а у конкретного user_id мало просмотренных мемов, модель может плохо предсказывать
• Целевая метрика precision = TP/P = TP/(TP+FP). Accuracy тут не нужно.
• Имеет смысл сделать отдельную рекомендательную систему для новых юзеров, которые просмотрели мало мемов.

## Тесты

Дается список из n юзеров, надо предсказать m мемов для каждого в формате csv с колонками user_id, meme_id. После чего они просматривают эти мемы и считается средний лайк-рейт, который сравнивается с текущей рекомендательной системой.

How many memes did user get?

In [16]:
event[['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).head(15)

,reaction_id
user_id,
81745542,47066
366481144,37491
431589633,36363
420302986,33748
376084050,27982
1254871249,27550
211921528,23956
936739890,23518
248960545,23394


# English users analysis

In [17]:
user_en = user.loc[user.interface_lang == 'en']['user_id']
event_en = event[event.user_id.isin(user_en)][['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).rename(columns = {'reaction_id':'count'})

In [18]:
event_en.head(5)

,count
user_id,
431589633,36363
941088926,4605
1150038020,3951
544090166,3871
1153477714,3493


In [33]:
print('Standard deviation: ', event_en.std()[0])
print('Median: ', event_en.median()[0])
print('Mean of memes number: ', event_en.mean()[0])

truncation = len(event_en)//10 #10% trim
print('Trimmed mean: ', event_en[truncation:][::-1][truncation:].mean()[0])

Standard deviation:  517.7767110304451
Median:  10.0
Mean of memes number:  47.53766279479056
Trimmed mean:  14.84469863616366


In [20]:
event_en.value_counts().head(15)

count
3        356
4        329
6        314
5        304
7        287
2        260
0        235
8        222
1        218
9        186
11       182
10       170
13       154
12       149
14       125
dtype: int64

# Russian users analysis

In [21]:
user_ru = user.loc[user.interface_lang == 'ru']['user_id']
event_ru = event[event.user_id.isin(user_ru)][['reaction_id', 'user_id']].groupby('user_id').count().sort_values('reaction_id', ascending = False).rename(columns = {'reaction_id':'count'})

In [22]:
event_ru.head(5)

,count
user_id,
81745542,47066
366481144,37491
420302986,33748
376084050,27982
1254871249,27550


In [34]:
print('Standart deviation: ', event_ru.std()[0])
print('Median: ', event_ru.median()[0])
print('Mean of memes number: ', event_ru.mean()[0])

truncation = len(event_ru)//10 #10% trim
print('Trimmed mean: ', event_ru[truncation:][::-1][truncation:].mean()[0])

Standart deviation:  1154.2435448537656
Median:  28.0
Mean of memes number:  235.66188334404225
Trimmed mean:  64.60096368341215


# Users leave analysis

In [44]:
event['created_at'] = pd.to_datetime(event['created_at'])
event['updated_at'] = pd.to_datetime(event['updated_at'])

In [72]:
dif = event['updated_at'] - event['created_at']
time_dif = dif.apply(lambda x: x.total_seconds())

In [78]:
event['time_difference'] = time_dif